# Generating object masks from input prompts with SAM


In [1]:
# %pip install segment-geospatial

In [1]:
import os
import leafmap
from samgeo import SamGeo, tms_to_geotiff

## Create an interactive map

In [2]:
m = leafmap.Map(center=[48.367462,10.939471], zoom=18, height="800px")
m.add_basemap("SATELLITE")
m

Map(center=[48.367462, 10.939471], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…

## Download a sample image

Pan and zoom the map to select the area of interest. Use the draw tools to draw a polygon or rectangle on the map

In [3]:
bbox = m.user_roi_bounds()
if bbox is None:
    bbox = [10.939471 - 0.001, 48.367462 - 0.001, 10.939471 + 0.001, 48.367462 + 0.001]

In [4]:
image = "Imagery/satellite_prompts.tif"
tms_to_geotiff(output=image, bbox=bbox, zoom=18, source="Satellite", overwrite=True)

Downloaded image 1/6
Downloaded image 2/6
Downloaded image 3/6
Downloaded image 4/6
Downloaded image 5/6
Downloaded image 6/6
Saving GeoTIFF. Please wait...
Image saved to Imagery/satellite_prompts.tif


You can also use your own image. Uncomment and run the following cell to use your own image.

In [5]:
# image = '/path/to/your/own/image.tif'

Display the downloaded image on the map.

In [5]:
m.layers[-1].visible = False
m.add_raster(image, layer_name="Image")
m

Map(bottom=23225670.0, center=[48.367462, 10.939471], controls=(ZoomControl(options=['position', 'zoom_in_text…

## Initialize SAM class

Specify the file path to the model checkpoint. If it is not specified, the model will to downloaded to the working directory.

Set `automatic=False` to disable the `SamAutomaticMaskGenerator` and enable the `SamPredictor`.

In [6]:
sam = SamGeo(
    model_type="vit_h",
    automatic=False,
    sam_kwargs=None,
)

Specify the image to segment. 

In [7]:
sam.set_image(image)

## Image segmentation with input points

A single point can be used to segment an object. The point can be specified as a tuple of (x, y), such as (col, row) or (lon, lat). The points can also be specified as a file path to a vector dataset. For non (col, row) input points, specify the `point_crs` parameter, which will automatically transform the points to the image column and row coordinates.

Try a single point input:

In [8]:
point_coords = [[48.367462,10.939471]]
sam.predict(point_coords, point_labels=1, point_crs="EPSG:4326", output="data/mask_point_test.tif")
m.add_raster("data/mask_point_test.tif", layer_name="Mask1", nodata=0, cmap="Blues", opacity=1)
m

No valid pixel coordinates found.


IndexError: index 0 is out of bounds for axis 0 with size 0

Try multiple points input:

In [10]:
point_coords = [[-122.1464, 37.6431], [-122.1449, 37.6415], [-122.1451, 37.6395]]
sam.predict(point_coords, point_labels=1, point_crs="EPSG:4326", output="mask2.tif")
m.add_raster("mask2.tif", layer_name="Mask2", nodata=0, cmap="Greens", opacity=1)
m

Map(bottom=3246791.0, center=[37.63844999999999, -122.13499999999999], controls=(ZoomControl(options=['positio…

## Interactive segmentation

Display the interactive map and use the marker tool to draw points on the map. Then click on the `Segment` button to segment the objects. The results will be added to the map automatically. Click on the `Reset` button to clear the points and the results.

In [9]:
m = sam.show_map()
m

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

In [ ]:
m = sam.show_map()
m

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…